# Prepare Data for Label Studio Import

This script prepares data for import into Label Studio. It converts YOLO labels into the Label Studio JSON format and creates a JSON file that can be imported into Label Studio.

## Usage

1. Place your YOLO labels and images in the same directory.
2. Create a `classes.txt` file containing the names of the classes in your dataset, one per line.
3. Run the notebook, changing the directory paths as needed.

### Launch Label Studio

In [ ]:
%%sh
pip install label-studio
label-studio

## Convert YOLO Labels into Label Studio Format
Existing YOLO labels are generated in the YOLO txt specification. This will convert them into the Label Studio JSON format.

In [ ]:
import os
import json

def yolo_to_label_studio(yolo_dir, output_json, classes_file):
    with open(classes_file, 'r') as f:
        class_names = f.read().strip().splitlines()
    
    result = []
    for label_file in os.listdir(yolo_dir):
        if label_file.endswith('.txt'):
            base_name = os.path.splitext(label_file)[0]
            # Create the image URL
            image_url = f"http://localhost:8081/{base_name}.png"
            label_path = os.path.join(yolo_dir, label_file)

            with open(label_path, 'r') as f:
                annotations = []
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.split())
                    annotations.append({
                        "from_name": "label",
                        "to_name": "image",
                        "type": "rectanglelabels",
                        "value": {
                            "x": (x_center - width / 2) * 100,
                            "y": (y_center - height / 2) * 100,
                            "width": width * 100,
                            "height": height * 100,
                            "rectanglelabels": [class_names[int(class_id)]]
                        }
                    })

                result.append({
                    "data": {"image": image_url},  # Use URL instead of local file path
                    "annotations": [{"result": annotations}]
                })

    with open(output_json, 'w') as f:
        json.dump(result, f, indent=4)

# Paths
yolo_dir = os.path.expanduser("~/Desktop/results/labels")
output_json = os.path.expanduser("~/Desktop/results/label-studio.json")
classes_file = os.path.expanduser("~/Desktop/results/classes.txt")

yolo_to_label_studio(yolo_dir, output_json, classes_file)

## Host HTTP Server Locally for Image Files

In [ ]:
%%capture
%pip install Flask Flask-CORS

In [ ]:
from flask import Flask, send_from_directory
from flask_cors import CORS
import threading
import os

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Define the directory to serve files from
directory_to_serve = os.path.expanduser("~/Desktop/results")

@app.route('/<path:filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(directory_to_serve, filename)

# Function to run the server
def run_server():
    app.run(host='0.0.0.0', port=8081)  # Bind to all interfaces

# Start the server in a new thread
threading.Thread(target=run_server).start()